# TRABALHO #1 — Grupo 6

 **Tópico: Informed and adversarial search strategies**

Membros:

 - Mariana Filipa Ribeiro Ferreira (up202205006)
 - Raquel Barbosa Ferreira Alves (up202206827)
 - Gonçalo Reimão Silva da Luz (up202205522)



## A* Algorithm

In [ ]:
def evaluate_segment(segment, player):  # heuristic
    num_player = segment.count(player)
    num_opponent = segment.count("🔴")

    if num_player == 0:
        if num_opponent == 3:
            return -50
        elif num_opponent == 2:
            return -10
        elif num_opponent == 1:
            return -1
        elif num_opponent == 4:
            return -999
        else:
            return 0
    elif num_opponent == 0:
        if num_player == 3:
            return 50
        elif num_player == 2:
            return 10
        elif num_player == 1:
            return 1
        elif num_player == 4:
            return 999
        else:
            return 0
    else:
        return 0

Esta função serve para implementar a heurística fornecida pelo professor ao receber dois parâmetros: 'segment', que é uma sequência de elementos representando um segmento do jogo, e 'player', que indica o jogador atual. A função avalia o segmento com base nas peças do jogador e do oponente presentes nele. Ela conta o número de peças do jogador ('num_player') e do oponente ('num_opponent') no segmento e atribui cotação tendo em conta isso.

In [ ]:
def evaluate_board(board, player):
    total_score = 0
    # opponent = "🔴" if player == "🔵" else "🔵"

    # Evaluate rows
    for row in board:
        for c in range(7 - 3):
            segment = row[c : c + 4]
            total_score += evaluate_segment(segment, player)

    # Evaluate columns
    for c in range(7):
        column = [board[r][c] for r in range(6)]
        for r in range(6 - 3):
            segment = column[r : r + 4]
            total_score += evaluate_segment(segment, player)

    # Evaluate diagonals
    for r in range(6 - 3):
        for c in range(7 - 3):
            segment = [board[r + i][c + i] for i in range(4)]
            total_score += evaluate_segment(segment, player)

    for r in range(6 - 3):
        for c in range(3, 7):
            segment = [board[r + i][c - i] for i in range(4)]
            total_score += evaluate_segment(segment, player)

    return total_score

A função "evaluate_board()" recebe dois inputs: o tabuleiro de jogo, e a ficha do jogador, que pode ser 🔴 ou 🔵 (No entanto, na implementação do algoritmo, esta função recebe sempre a ficha de jogo "🔵" representante do jogador AI como input). Esta função retorna a pontuação total para o jogador em questão.

Esta função itera por todos os segmentos possiveis numa linha, coluna, e diagonal do tabuleiro (de modo semelhante à função "check_winner()" descrita acima) e avalia-os através da função "evaluate_score()", determinando assim uma pontuação para o jogador.

In [ ]:
def A_star_move(board):

    possible_moves = [col for col in range(7) if board[0][col] == "🔲"]
    best_move = possible_moves[0]
    best_score = -float("inf")

    for move in possible_moves:
        temp_board = [row[:] for row in board]
        drop_chip(temp_board, move, "🔵")
        total_score = evaluate_board(temp_board, "🔵")
        if total_score > best_score:
            best_score = total_score
            best_move = move

    return best_move

# from game
def drop_chip(board, column, chip):
    for row in range(5, -1, -1):
        if board[row][column] == "🔲":
            board[row][column] = chip
            return True
    return False

Esta função é chamada diretamente pelo "main()". A função "A_star_move()" recebe o tabuleiro e, utilizando as funções auxiliares descritas acima, seleciona a melhor jogada para o AI.

Primeiro, esta função identifica todos os movimentos possiveis utilizando a função "drop_chip()". Esta função recebe como input o tabuleiro (uma lista), uma coluna do tabuleiro, e uma ficha de jogo (🔴 & 🔵). Esta função tenta colocar uma determinada ficha de jogo no tabuleiro. Se esse movimento for possível, a função retorna Tryue. Se não for possível (caso, por exemplo, a coluna selecionada esteja cheia), a função retorna False.

De seguida, itera-se sobre cada movimento possível, e simula a colocação de uma ficha na coluno num tabuleiro temporário. Esse tabuleiro é avaliado usando a função "evaluate_board()" e o movimento que será concretizado no tabuleiro real é o movimento que obtiver a maior pontuaçãp, sendo retornado o número da coluna correspondente.

## Monte-Carlo Tree Search

In [ ]:
import copy
import math
import random

LINES = 6
COLUMNS = 7
AI_CHIP = "🔵"
PLAYER_CHIP = "🔴"


class Node:
    def __init__(self, estado, pai=None):
        self.estado = estado
        self.pai = pai
        self.filhos = []
        self.visitas = 0
        self.vitorias = 0
        self.score = 0

Neste excerto de código começamos por dar "import" às bibliotecas necessárias (copy, math, e random), seguidas das constantes necessárias para facilitar o código. As variáveis LINES e COLUMNS definem o tamanho do tabuleiro do jogo e AI_CHIP e PLAYER_CHIP para representarem as fichas do jogador e do adversário (🔵 & 🔴 respetivamente).

Aqui também é definida a classe Node, que representa um nó na árvore de busca. Cada nó armazena um estado do jogo, uma referência ao nó pai, uma lista de nós filhos, o número de visitas ao nó, o número de vitórias a partir do nó e um score (pontuação) calculado.

In [ ]:
def check_winner(board, chip):
    # 4 EM LINHA HORIZONTAL
    for c in range(7 - 3):  # Iterate over columns
        for r in range(6):  # Iterate over rows
            if (
                board[r][c] == chip
                and board[r][c + 1] == chip
                and board[r][c + 2] == chip
                and board[r][c + 3] == chip
            ):
                return True

    # 4 EM LINHA VERTICAL
    for c in range(7):  # Iterate over columns
        for r in range(6 - 3):  # Iterate over rows
            if (
                board[r][c] == chip
                and board[r + 1][c] == chip
                and board[r + 2][c] == chip
                and board[r + 3][c] == chip
            ):
                return True

    # 4 EM LINHA DIAGONAL (pos)
    for c in range(7 - 3):  # Iterate over columns
        for r in range(6 - 3):  # Iterate over rows
            if (
                board[r][c] == chip
                and board[r + 1][c + 1] == chip
                and board[r + 2][c + 2] == chip
                and board[r + 3][c + 3] == chip
            ):
                return True

    # 4 EM LINHA DIAGONAL (neg)
    for c in range(7 - 3):  # Iterate over columns
        for r in range(3, 6):  # Iterate over rows
            if (
                board[r][c] == chip
                and board[r - 1][c + 1] == chip
                and board[r - 2][c + 2] == chip
                and board[r - 3][c + 3] == chip
            ):
                return True

    return False

A função "check_winner()" serve para verificar se há um vencedor do jogo, ou seja, se existem 4 (ou mais) fichas de jogo em posições consecutivas, seja na horizontal, vertical, ou na diagonal. Esta função recebe como input uma lista que contém a representação do tabuleiro de jogo, e uma determinada ficha de jogo que corresponde a um dos dois jogadores (🔴 & 🔵). Se existir uma posição de vitória para a ficha indicada, a função retorna True. Se não, retorna False.

In [ ]:
def is_board_full(board):
    # Função explicada no interface
    for row in board:
        if "🔲" in row:
            return False
    return True

def terminal(estado):
    # Ver se alguém ganhou
    return check_winner(estado, AI_CHIP) or check_winner(estado, PLAYER_CHIP) or is_board_full(estado)


def movimentos_validos(estado):
    # Retorna uma lista de colunas válidas onde um chip pode ser colocado
    valid_moves = []
    for coluna in range(COLUMNS):
        if estado[0][coluna] == "🔲":
            valid_moves.append(coluna)
    return valid_moves


def proxima_linha_disponivel(estado, coluna):
    # Encontra a próxima linha válida numa coluna
    for linha in range(LINES-1, -1, -1):
        if estado[linha][coluna] == "🔲":
            return linha
    return None


def resultado(estado):
    if check_winner(estado, AI_CHIP):
        return "win"  # AI ganha
    elif check_winner(estado, PLAYER_CHIP):
        return "lose"  # Oponente ganha
    else:
        return "draw"  # Empate

Estas são umas pequenas funções auxiliares necessárias para o funcionamento do código:
- 'terminal' server para verificar se o jogo acabou, ou seja se alguém ganhou ou se o tabuleiro ficou cheio.  
- 'movimentos_validos' retorna uma lista com todos os locais onde será possível jogar.
- 'proxima_linha_disponivel' retorna a linha que esteja vazia ao longo do tabuleiro.
- 'resultado' verifica quem ganhou ou se houve um empate.

In [ ]:
def expandir(node):
    player = aux_expandir(node)
    for i in movimentos_validos(node.estado):
        linha = proxima_linha_disponivel(node.estado, i)
        estado_next = copy.deepcopy(node.estado)
        estado_next[linha][i] = player
        next_node = Node(estado_next)
        next_node.pai = node
        node.filhos.append(next_node)

        
def aux_expandir(node):
    n_ai = 0
    n_player = 0
    for line in node.estado:
        for element in line:
            if element == AI_CHIP:
                n_ai += 1
            elif element == PLAYER_CHIP:
                n_player += 1
    if n_ai == n_player:
        return PLAYER_CHIP
    elif n_ai < n_player:
        return AI_CHIP

A função 'expandir' é responsável por expandir um nó na árvore de busca. Ela primeiro chama aux_expandir(node) para determinar qual jogador fará o próximo movimento com base no estado atual do tabuleiro representado pelo nó. Em seguida, itera sobre os movimentos válidos disponíveis no estado atual do tabuleiro. Para cada movimento válido, determina a próxima linha disponível na coluna selecionada. Em seguida, cria um novo estado do tabuleiro copiando o estado atual e fazendo a jogada na próxima linha disponível. Com base neste novo estado, cria um novo nó na árvore (next_node) e o adiciona como filho do nó atual (node).

A 'aux_expandir' conta o número de fichas do jogador AI e do jogador adversário no tabuleiro para perceber quem tem de fazer o próximo movimento, sendo que nunca é possível haverem mais chips 🔵 do que 🔴.

In [ ]:
def find_leafs(node):
    if not node.filhos:
        return [node]
    leafs = []
    for filho in node.filhos:
        leafs.extend(find_leafs(filho))
    return leafs


def select_node_to_expand(tree):
    leaf_nodes = find_leafs(tree)
    ucb1_best = -float("inf")
    node_to_expand = None
    for node in leaf_nodes:
        player = aux_expandir(node)
        if terminal(node.estado) == False:
            if node.visitas == 0:
                return node  # Tratar nós não visitados como tendo um valor UCB1 alto
            elif player == AI_CHIP:
                ucb1 = node.vitorias / node.visitas + 0.8 *math.sqrt( math.log(node.pai.visitas) / node.visitas)
            elif player == PLAYER_CHIP:
                ucb1 = (1 - (node.vitorias / node.visitas)) + 0.8 *math.sqrt( math.log(node.pai.visitas) / node.visitas)
            if ucb1 > ucb1_best:
                node_to_expand = node
                ucb1_best = ucb1
    return node_to_expand

A função 'find_leafs' serve para encontrar todos os nós folhas, ou seja, os nós que não tiverem filhos.

De seguida, a próxima função é responsável por selecionar um nó para expandir na árvore de busca, usando o critério Upper Confidence Bound 1 (UCB1). O UCB1 é uma combinação de duas partes: uma estimativa da média das vitórias sobre visitas e uma medida de exploração que é proporcional à raiz quadrada do logaritmo do número de visitas do nó pai sobre o número de visitas do nó atual. Se o jogador associado ao nó for o jogador AI, é maximizado o UCB1 para explorar mais jogadas favoráveis. Caso contrário, é maximizado o UCB1 para explorar jogadas que resultem em menos vitórias para o adversário. O nó com o maior valor UCB1 é selecionado para expansão e retornado como resultado.

In [ ]:
def simulacao(estado, jogador_atual):
    estado_simulado = copy.deepcopy(estado)
    while not terminal(estado_simulado):
        movimentos_possiveis = movimentos_validos(estado_simulado)
        movimento = random.choice(movimentos_possiveis)
        linha = proxima_linha_disponivel(estado_simulado, movimento)
        estado_simulado[linha][movimento] = jogador_atual
        jogador_atual = PLAYER_CHIP if jogador_atual == AI_CHIP else AI_CHIP
    return resultado(estado_simulado)


def retropropagacao(node, resul):
    while node is not None:
        node.visitas += 1
        if resul == "win":
            node.score += 1
            node.vitorias += 1
        elif resul == "lose":
            node.score -= 1
        elif resul == "draw":
            node.score += 0
        node = node.pai

Na 'simulacao' durante cada iteração do loop, que só termina quando a codição da função 'terminal' é verificada, é selecionado aleatoriamente um movimento válido para o jogador atual, de seguida atualiza o estado do tabuleiro com o movimento escolhido e alterna para o próximo jogador. Ao final, retorna o resultado do jogo a partir do estado final do tabuleiro.

A 'retropropagação' é responsável por atualizar as estatísticas dos nós na árvore de busca com base no resultado da simulação. Ela percorre o caminho de nós do nó folha até a raiz, incrementando o número de visitas de cada nó e ajustando as contagens de vitória, derrota ou empate, dependendo do resultado da simulação.

In [ ]:
def selecionar_melhor_filho(node):
    melhor_filho = None
    super_max = -float('inf')
    score_max = -float('inf')
    for filho in (node.filhos):
        valor = filho.vitorias/filho.visitas
        if valor > super_max:
            if filho.score > score_max:
                super_max = valor
                melhor_filho = filho
                score_max = filho.score
    return melhor_filho

Aqui selecionamos o melhor filho de um nó na árvore de busca, com base nas estatísticas acumuladas durante as simulações, tendo em conta a proporção de vitórias sobre visitas e o que tiver maior score, para ter em conta o filho com mais vitórias e menos derrotas. 

In [ ]:
def comparar_tab(atual,futuro):
    for c in range (7):
        for r in range (6):
            if atual[r][c] != futuro[r][c]:
                return c
            
def MCTS_move(tabuleiro):
    estado_raiz = copy.deepcopy(tabuleiro)
    node_raiz = Node(estado_raiz)
    player = AI_CHIP

    for i in range(500):  # número de simulações (alterar para obter resultados diferentes)
        node_a_expandir = select_node_to_expand(node_raiz)
        if node_a_expandir == None:
            break
        expandir(node_a_expandir)
        for node in node_a_expandir.filhos:
            player = aux_expandir(node)
            for _ in range (5):
                resultado_simulacao = simulacao(node.estado, player)
                retropropagacao(node, resultado_simulacao)
        print(i)
    melhor_filho = selecionar_melhor_filho(node_raiz)
    return comparar_tab(tabuleiro, melhor_filho.estado)

A função "MCTS_move()" é a principal função deste método de procura, que é chamada pelo interface do jogo. É onde é decidida a jogada a ser realizada pelo AI através das várias funções auxiliares. Começamos por criar uma  cópia do estado atual do tabuleiro para evitar modificar o estado original durante a busca seguido da criação de um nó raiz da árvore de busca com o estado do tabuleiro atual.

No loop principal é decidido o número de iterações a serem realizadas do algoritmo MCTS, nós optamos por 500 que pareceu o número mais razoável para o tempo de espera. Durante cada iteração seleciona um nó para expandir usando a função select_node_to_expand(node_raiz). Expande o nó selecionado adicionando seus filhos e realiza uma simulação a partir de cada filho do nó expandido para obter um resultado de simulação para não percorrer sempre o mesmo node. O print(i) serve apenas para ter noção do tempo de espera.

Após as simulações, seleciona o melhor filho do nó raiz com base nas estatísticas acumuladas durante as simulações, por fim retorna o próximo movimento a ser feito pela AI, comparando o estado do tabuleiro atual com o estado do melhor filho selecionado.


## MinMax

In [ ]:
import random
import math

ROW_COUNT = 6
COLUMN_COUNT = 7

PLAYER = 0
AI = 1

EMPTY = "🔲"
PLAYER_PIECE = "🔴"
AI_PIECE = "🔵"

WINDOW_LENGTH = 4

def create_board():
    board = [[EMPTY] * COLUMN_COUNT for _ in range(ROW_COUNT)]
    return board

Começamos por importar as bibliotecas "random" para inicializar colunas aleatoriamente e "math" para operações matemáticas. Definimos também valores fixos, como o número de linhas, o número de colunas, os valores atribuídos ao jogador e à AI, as peças que compõe o tabuleiro e o comprimento de um segmento relevante (4 peças seguidas).
A função create_board cria um tabuleiro vazio.

In [ ]:
def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROW_COUNT - 1][col] == EMPTY

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == EMPTY:
            return r

def print_board(board):
    print("==============")
    for row in reversed(board):
        print(' '.join(row))

A função "drop_piece" coloca a peça desejada (🔴 ou 🔵) no respetivo lugar do tabuleiro, substituindo o espaço vazio.
Em "is_valid_location" uma certa coluna do tabuleiro é avaliada para verificar se é possível colocar lá uma peça. Para isso, basta confirmar que a posição no topo da coluna está vazia.
Para determinar em que linha a peça será inserida, usamos "get_next_open_row", que retorna a próxima linha disponível (não preenchida).
A função "print_board" permite uma visualização do tabuleiro, imprimindo-o na saída padrão. Ela funciona iterando sobre as linhas do tabuleiro, mas no sentido invertido, ou seja, começa a imprimir da última linha até a primeira.



In [ ]:
def winning_move(board, piece):
    # Horizontais
    for c in range(COLUMN_COUNT - 3):
        for r in range(ROW_COUNT):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True

    # Verticais
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT - 3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True

    # Diagonais positivas
    for c in range(COLUMN_COUNT - 3):
        for r in range(ROW_COUNT - 3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True

    # Diagonais negativas
    for c in range(COLUMN_COUNT - 3):
        for r in range(3, ROW_COUNT):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True


Nesta função, verificamos todos os segmentos de 4 peças seguidas, quer seja na horizontal, vertical ou diagonais. Ela retorna True se existirem 4 peças do mesmo jogador no segmento.

In [ ]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE
    
    if window.count(piece) == 4:
        score += 90000
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 50
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 10
    elif window.count(piece) == 1 and window.count(EMPTY) == 3:
        score += 1

    if window.count(opp_piece) == 4:
        score -= 10000
    elif window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 50
    elif window.count(opp_piece) == 2 and window.count(EMPTY) == 2:
        score -= 10
    elif window.count(opp_piece) == 1 and window.count(EMPTY) == 3:
        score -= 1

    return score

Aqui são definidos os valores para cada situação que pode ocorrer num segmento. Por exemplo, quando a jogada resulta em 4 peças seguidas, o valor atribuído é 90000, enquanto que se forem 3 o valor é 50. Caso as peças sejam do oponente, o valor é negativo.

In [ ]:
def score_position(board, piece):
    score = 0

    # Centro
    center_array = [board[i][COLUMN_COUNT // 2] for i in range(ROW_COUNT)]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Horizontal
    for r in range(ROW_COUNT):
        row_array = board[r]
        for c in range(COLUMN_COUNT - 3):
            window = row_array[c:c + WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Vertical
    for c in range(COLUMN_COUNT):
        col_array = [board[i][c] for i in range(ROW_COUNT)]
        for r in range(ROW_COUNT - 3):
            window = col_array[r:r + WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Diagonal positiva
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + i][c + i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    # Diagonal negativa
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + 3 - i][c + i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    return score

Em "score_position", a situação atual do tabuleiro é avaliada e são atribuídos valores por cada segmento de 4 usando a função evaluate_window. Desta forma, o estado atual recebe um valor que poderá ser comparado com outros estados e determinar se é melhor ou pior.

In [ ]:
def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMN_COUNT):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

def pick_best_move(board, piece):
    valid_locations = get_valid_locations(board)
    best_score = -10000
    best_col = random.choice(valid_locations)
    for col in valid_locations:
        row = get_next_open_row(board, col)
        temp_board = [row[:] for row in board]
        drop_piece(temp_board, row, col, piece)
        score = score_position(temp_board, piece)
        if score > best_score:
            best_score = score
            best_col = col

    return best_col

A função "get_valid_locations" retorna uma lista com as jogadas válidas possíveis.
Em "pick_best_move" verifica-se qual é a jogada ideal usando um tabuleiro temporário e substituindo best_score por score, caso este seja mais alto para aquela coluna.

In [ ]:
def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(get_valid_locations(board)) == 0

def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER_PIECE):
                return (None, -10000000000000)
            else:  # Game is over, no more valid moves
                return (None, 0)
        else:  # Depth is zero
            return (None, score_position(board, AI_PIECE))
    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = [row[:] for row in board]
            drop_piece(b_copy, row, col, AI_PIECE)
            new_score = minimax(b_copy, depth - 1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value

    else:  # Minimizing player
        value = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = [row[:] for row in board]
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth - 1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

Na primeira função, "is_terminal_node", verificamos se o estado do tabuleiro é final, ou seja, se o jogo já acabou. Isto acontece caso um jogador ganhe ou já não hajam mais espaços para inserir as peças.
Na função "minimax", primeiro obtemos as colunas válidas e verificamos se o estado do tabuleiro é final. Se for final, a função não retorna nenhuma coluna. Caso contrário, irá passar por uma sequência de testes. Em suma, a função ramifica o estado atual do tabuleiro em tantos ramos quanto o número de colunas válidas. Cada ramificação também se divide, até chegar à profundidade desejada. Assim que isto acontece, cada conjunto de ramificações é comparado entre eles e é escolhido o melhor dependendo de quem está a escolher. Se for "maximizingPlayer", este irá querer escolher o tabuleiro com a pontuação mais alta. No entanto, se for o "minimizingPlayer", ele escolherá o que tem a pontuação mais baixa (representando a jogada do oponente). Desta forma, consegue prever um grande número de jogadas e escolher a coluna mais favorável ao estado atual.

In [ ]:
def move(human, ai):
	board = create_board()
	print_board(board)
	game_over = False

	turn = 0

	while not game_over:

		if turn == PLAYER and not game_over:
			col = input("Escolhe uma coluna de 1 a 7: ")
			if col == 'x':
				print("FORCE-QUITTING GAME. . .")
				break

			if col.isdigit():
				col = int(col)
				col -= 1

			# Ask for Player 1 Input
			if is_valid_location(board, col):
				row = get_next_open_row(board, col)
				drop_piece(board, row, col, PLAYER_PIECE)

				if winning_move(board, PLAYER_PIECE):
					game_over = True

				turn += 1
				turn = turn % 2

				print_board(board)

			else:
				print("Escolha inválida!")

		
		if turn == AI and not game_over:
			col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)

			if is_valid_location(board, col):
				# pygame.time.wait(500)
				row = get_next_open_row(board, col)
				drop_piece(board, row, col, AI_PIECE)

				if winning_move(board, AI_PIECE):
					game_over = True

				print_board(board)

				turn += 1
				turn = turn % 2

		if game_over:
			if winning_move(board, PLAYER_PIECE):
				print(f"{human} vence!")
			else:
				print(f"{ai} vence!")

			print("FORCE-QUITTING GAME. . .")

Esta é a função principal deste algoritmo. Permite ao utilizador interagir com o tabuleiro, e chama a função minimax para determinar a melhor coluna possível. Também imprime os vencedores e as instruções do jogo.

## Game Interface

In [ ]:
import time

# o módulo time serve para que a jogada do A* não seja automática fazendo com que o jogo fique menos confuso no terminal pq JESUS

def print_board(board):
    for row in board:
        print(" ".join(row))


def drop_chip(board, column, chip):
    for row in range(5, -1, -1):
        if board[row][column] == "🔲":
            board[row][column] = chip
            return True
    return False


def is_board_full(board):
    for row in board:
        if "🔲" in row:
            return False
    return True

Esta secção do interface do jogo 4-em-linha começa com "import time", que é exclusivamente utilizado na função "main()" para fazer com que o programa espere 0.5 segundos depois de uma jogada. Isto serve simplesmente para fazer com que o terminal fique menos confuso com as sucessivas "impressões".

De seguida temos 3 funções implementadas, uma dais quais já foi explicada acima (a função "drop_chip()"). A função "print_board()" recebe uma lista como input e imprime-o para a saída padrão no terminal, representando assim o estado atual do tabuleiro. A função "is_board_full(board)" recebe a lista representante do tabuleiro e verifica se existe o símbolo 🔲, o que indica um espaço livre no tabuleiro. Se este símbolo existir, significa que o tabuleiro não está cheio, logo a função retorna False. Caso contrário, retorna True.

In [ ]:
def main():
    board = [["🔲"] * 7 for _ in range(6)]
    print("4 em Linha")

    player_one = input("Nome do Jogador: ")
    #   player_two = input("Nome do JOgador 2: ")
    player_two = "AI"

    while True:
        print("==============")
        print("Algoritmo 'a' - A*")
        print("Algoritmo 'b' - Monte-Carlo Tree Search")
        print("Algoritmo 'c' - MinMax (IN TESTING)")
        choice = input("Pretende jogar contra qual algoritmo? ")
        time.sleep(0.5)
        
        if choice == 'a':
            print("Selecionou o algoritmo A*.")
            break
        elif choice == 'b':
            print("Selecionou o algoritmo Monte-Carlo Tree Search.")
            break
        elif choice == 'c':
            print("Selecionou o algoritmo MinMax.")
            time.sleep(0.5)
            print("Este algoritmo apenas sai do jogo com um FORCE-QUIT. . .")
            time.sleep(0.5)
            move(player_one, player_two)
            time.sleep(0.5)
            column == 'x'
            break
        else:
            print("Escolha inválida!")

    print("%s contra %s" % (player_one, player_two))
    print("==============")
    print_board(board)
    print("%s controla 🔴" % player_one)
    print("AI controla 🔵")

    players = [(player_one, "🔴"), (player_two, "🔵")]
    turn = 0

    while True:
        player, chip = players[turn % 2]

        if player == player_one:
            while True:
                column = input("Escolhe uma coluna de 1 a 7: ")
                if column == 'x':
                    print("FORCE-QUITTING GAME. . .")
                    break
                if column.isdigit() and 1 <= int(column) <= 7:
                    column = int(column) -1
                    break
                else:
                    print("Escolha inválida!")

            print("==============")
            time.sleep(0.5)
        else:
            print("É a vez do %s." % (player_two))
            print("==============")

            if choice == 'b':
                column = MCTS_move(board)
                # time.sleep(0.5)
            else:
                column = A_star_move(board)
                time.sleep(0.5)

        if column is not None and 0 <= column < 7:
            if drop_chip(board, column, chip):
                print_board(board)
                if check_winner(board, chip):
                    print(f"{player} vence!")
                    break
                elif is_board_full(board):
                    print("Empate!")
                    break
                turn += 1
            else:
                print("Escolha inválida!")
        else:
            print("Escolha inválida!")


if __name__ == "__main__":
    main()

A função "main()" é a função principal que cria o ambiente de jogo. Inicialmente, define a variável "board" como sendo uma matrix (lista de listas) de 6 linhas e 7 colunas, completamente preenchida com o símbolo 🔲, que reresentam os espaços vazios.

De seguida, solicita o nome do jogador humano, que é sempre o primeiro jogador, representado com a ficha de jogo azul (🔵), e define de seguida o segundo jogador como sendo o "AI", representado com a ficha de jogo vermelha (🔴).

Após a definição do jogo, um loop "while" permite que o jogador escolha contra qual algortimo pretende jogar. Selecionando a opção "a", o AI será controlado pelo algoritmo A*. Selecionando a opção "b", o AI será controlado pelo algoritmo Monte-Carlo Tree Search. Esta escolha é definida pela variável "choice". Selecionando o algoritmo "c", o AI será controlado pelo algoritmo MinMax (este algoritmo não tem uma saída suave).

Depois dessa escolha, existe outro loop "while". Iniciando-se com o jogador humano, o programa solicita o input de uma coluna para colocar a sua ficha de jogo. Para fins de "debugging", foi adicionada a possibilidade de fazer um "force-quit" do jogo, possivel pelo input the um "x" em vez de um inteiro de 1 a 7.

Depois da jogada, o tabuleiro é atualizado e é verificado se existe um vencedor ou empate. Se isso não acontecer, a ronda passa para o AI. De acordo com a escolha de algoritmo, a função "MCTS_move()" é chamada, ou a função "A_star_move()". Isto repete-se infinitamente até não existirem mais espaços "válidos" para inserir fichas de jogo (empate), ou até qualquer um dos jogadores vencer (vitória).